In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

all_teams = set()
all_voters = set()
tester = set()
def author_ballot_dictionary(Weeks, url, year):
    Dictionaries = []
    teams = set()
    for week in Weeks:
        new_url = url + week
        response = requests.get(new_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
        else:
            print("bad")
            continue
        parent_element = soup.find('div', id='gridBallots')
        Dict = {}
        if parent_element:
            Rows = parent_element.find_all('div', class_='gridRow')
            for row in Rows:
                author_name = (row.find('div', class_='gridPollster').find('a'))['href']
                correct_author_name = author_name.split("/")[3]
                Team_rows = row.find_all('div', class_= 'gridTeam')
                if Team_rows[0].get('class')[1][3:] == 'blank':
                    continue
                team_names = [correct_author_name, year, Weeks.index(week) + 1]
                for item in Team_rows:
                    correct_team_name = item.get('class')[1][3:]
                    team_names.append(correct_team_name)
                    all_teams.add(correct_team_name)
                Dict[correct_author_name] = team_names
                all_voters.add(correct_author_name)
        Dictionaries.append(Dict)

        df_file_path = "./raw_data/separate/season_" + year + "/week" + str(Weeks.index(week) + 1) + "_poll.csv"
        df = pd.DataFrame.from_dict(Dict, orient='index', columns=['Pollster', 'Season', 'Week', '1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th', '13th', '14th', '15th', '16th', '17th', '18th', '19th', '20th', '21st', '22nd', '23rd', '24th', '25th'])
        df.to_csv(df_file_path, mode='w', index=False, header=True)

        team_list_file = "./processed_data/separate_team_names/season_" + year + "/week" + str(Weeks.index(week) + 1) + "_top25.csv"
        ordered_team_list = team_names[3:]
        ordered_team_list.sort()
        write_to_file(team_list_file, ordered_team_list)

    return Dictionaries

def write_to_file(file_path, list_of_data):
    with open(file_path, 'w') as file:
        for data in list_of_data:
            file.write(data + "\n")

def csv_data_appender_by_year(year): 
    Weeks = ['pre-season', 'week-2', 'week-3', 'week-4', 'week-5', 'week-6', 'week-7', 'week-8', 'week-9', 'week-10', 'week-11', 'week-12', 'week-13', 'week-14', 'week-15', 'week-16', 'final-rankings']
    url = "https://collegepolltracker.com/football/grid/" + year + "/"
    Dictionaries = author_ballot_dictionary(Weeks, url, year)

    for dicto in Dictionaries:
        df = pd.DataFrame.from_dict(dicto, orient='index', columns=['Pollster', 'Season', 'Week', '1st', '2nd', '3rd', '4th', '5th', '6th', '7th', '8th', '9th', '10th', '11th', '12th', '13th', '14th', '15th', '16th', '17th', '18th', '19th', '20th', '21st', '22nd', '23rd', '24th', '25th'])
        df.to_csv('./raw_data/college_polls_original.csv', mode='a', index=False, header=True)


def csv_creation():
    years = ["2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024"]
    
    for year in years:
        csv_data_appender_by_year(year)

    all_voters_file = "./processed_data/all_names/all_voters.csv"
    all_teams_file = "./processed_data/all_names/all_ranked_teams.csv"
    
    ordered_all_teams = list(all_teams)
    ordered_all_voters = list(all_voters)
    ordered_all_teams.sort()
    ordered_all_voters.sort()
    
    write_to_file(all_voters_file, ordered_all_voters)
    write_to_file(all_teams_file, ordered_all_teams)

csv_creation()